# Instant Gratification!

Let's build a useful LLM solution - in a matter of minutes.

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, be sure to have followed the instructions in the "README" file, including creating your API key with OpenAI and adding it to the `.env` file.

In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [3]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
openai = OpenAI()

In [4]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Let's try one out

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [6]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [8]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [9]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [10]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nThe website introduces Edward Donner, a tech enthusiast and co-founder of Nebula.io, a company focused on leveraging AI for talent discovery and management. Edward shares his interests in coding, experimenting with large language models (LLMs), and producing electronic music. \n\n### Recent Posts\n1. **August 6, 2024** - *Outsmart LLM Arena*: A feature outlining a competitive environment where LLMs engage in strategic interactions involving diplomacy and manipulation.\n   \n2. **June 26, 2024** - *Choosing the Right LLM*: A guide that provides tools and resources for selecting the appropriate LLM for various uses.\n   \n3. **February 7, 2024** - *Fine-tuning an LLM on your texts*: Discusses methods for personalizing LLMs using one’s own texts.\n   \n4. **January 31, 2024** - *Fine-tuning an LLM on your texts: part 4 – QLoRA*: Continuation of the discussion about advanced fine-tuning techniques for LLMs.\n\nOverall, the site reflects Edward’s exp

In [11]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [12]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website serves as a personal space for sharing insights, experiments, and resources related to large language models (LLMs) and artificial intelligence. Ed introduces himself as a programmer and musician, and highlights his role as co-founder and CTO of Nebula.io, a company focused on leveraging AI to enhance talent discovery and engagement.

## Recent Posts
- **August 6, 2024**: An announcement about "Outsmart," an arena designed for LLMs to engage in diplomatic and strategic challenges.
- **June 26, 2024**: A post discussing tools and resources for selecting the right LLM.
- **February 7, 2024**: Exploration of fine-tuning LLMs based on personal text content.
- **January 31, 2024**: A continuation of the fine-tuning discussion with a focus on QLoRA.

The website emphasizes Ed's background in AI startups and his passion for coding and music, inviting others to connect and engage with him on these topics.

In [13]:
display_summary("https://cnn.com")

# Summary of CNN's Website

CNN is a leading news platform that provides the latest updates and in-depth analysis on a variety of topics including US and world news, politics, business, health, entertainment, and sports. The website features both video content and articles, offering a comprehensive source of breaking news.

## Recent Headlines:
- **2024 Presidential Race**: Coverage includes ongoing analysis of candidates like Trump and Harris, with speculation on their approaches leading up to debates and the election.
  
- **International Conflicts**: Reports of a significant Russian strike on a military facility in Ukraine and updates on the Israel-Hamas war, where public opinion dynamics are highlighted as a critical factor affecting leaders like Netanyahu.

- **Tragic Incidents**: Multiple stories regarding violence in the US including shootings in Washington state and a tragic bus crash in Mississippi affecting several individuals.

- **Cultural Notes**: Highlights from the entertainment world, including the passing of actor James Darren and notable events around the US Open tennis tournament.

- **Health and Science**: Articles discuss significant health topics, including recent analyses on ketamine and its implications.

CNN's platform also accentuates engagement with audiences through feedback tools and social media integration, aiming to foster a user-friendly and interactive news experience.

In [14]:
display_summary("https://anthropic.com")

# Summary of Anthropic Website

Anthropic is an AI safety and research company based in San Francisco, focused on developing reliable and beneficial AI systems. Their flagship AI model, **Claude 3.5 Sonnet**, is the latest version available for use, emphasizing enhanced intelligence and safety in AI interactions.

## Key Updates
- **New AI Model**: Claude 3.5 Sonnet released on **June 21, 2024**.
- **Research Announcements**:
  - **Constitutional AI: Harmlessness from AI Feedback** (Dec 15, 2022)
  - **Core Views on AI Safety**: Discussed principles of AI safety (Mar 8, 2023).

The company promotes building applications using their API to drive efficiency and innovate revenue streams. They also invite new talent to join their interdisciplinary team.